In [1]:
import numpy as np
import torch
import sys
import scipy
import pprint
from torch import nn, optim

In [2]:
sys.path.insert(0, 'GCN/pygcn')

In [3]:
from pygcn.utils import load_data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path='GCN/pygcn/data/cora/')

Loading cora dataset...


In [4]:
print(features.shape, adj.shape, labels.shape, torch.max(labels))

torch.Size([2708, 1433]) torch.Size([2708, 2708]) torch.Size([2708]) tensor(6)


# Matrix functions
**Note**: Most of these functions must be changed as soon as torch will provide an implementation for them. At the moment (torch 1.3) most of these sparse functions are unavailable or not-well documented, hence the necessity of implementation below (that may be unefficent or not 100% correct)

In [5]:
def sparse_ceil(torch_sparse_tensor: torch.sparse.FloatTensor) -> None:
    """
    In place ceil of torch sparse tensor values, must be changed when torch implementation is available
    """
    torch_sparse_tensor._values().ceil_()

In [92]:
def sparse_degree_matrix(A) -> torch.sparse.FloatTensor:
    D = scipy.sparse.spdiags(torch.sparse.sum(A, dim=1).to_dense(),0, *A.shape).tocoo()
    return coo_to_torch_sparse_matrix(D)

In [93]:
def coo_to_torch_sparse_matrix(coo):
    idx = torch.LongTensor(coo.nonzero())
    data = torch.FloatTensor(coo.data)
    return torch.sparse.FloatTensor(idx, data)

In [116]:
def laplacian(A, normalized=True):
    I = torch.eye(A.shape[0])
    D = sparse_degree_matrix(A)
    if not normalized:
        L = D.sub(A)
    else:
        D = (D**(-(1/2))).to_dense()
        p1 = torch.spmm(A, D)
        p2 = torch.mm(p1, D)
        L = coo_to_torch_sparse_matrix(scipy.sparse.coo_matrix(I - p2))

    return L

In [118]:
sparse_ceil(adj)
L = laplacian(adj)

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [   0,    8,   14,  ..., 1389, 2344, 2707]]),
       values=tensor([ 0.8333, -0.1667, -0.0500,  ..., -0.2000, -0.5000,
                       0.7500]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)



tensor(indices=tensor([[   0,    1,    2,  ..., 1389, 2344, 2707],
                       [   0,    1,    2,  ..., 2707, 2707, 2707]]),
       values=tensor([ 5.,  2.,  5.,  ..., -1., -1.,  3.]),
       size=(2708, 2708), nnz=15970, layout=torch.sparse_coo)
